## Vectorstores
Interfaces for specialized databases that can search over unstructured data with natural language

In [22]:
from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import pandas as pd 
import os
load_dotenv()
my_api_key = os.getenv("GOOGLE_API_KEY") 
genai.configure(api_key=my_api_key)

#### Chroma

```python 
            pip install langchain-chroma

In [25]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Load the document,  
raw_documents = TextLoader('Data/state_of_the_union.txt', encoding = 'utf-8').load()
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
# embed each chunk
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# load it into the vector store.
db = Chroma.from_documents(documents, embeddings)

In [26]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâs top legal minds, who will continue Justice Breyerâs legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâs been nominated, sheâs received a broad range of supportâfrom the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


In [27]:
found_docs =  db.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

1. One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâs top legal minds, who will continue Justice Breyerâs legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâs been nominated, sheâs received a broad range of supportâfrom the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâve set up joint patrols with Mexico and Guatemala to catch more human traffickers. 


##### Pinecone 

```python 
        pip install langchain-pinecone
```
###### Set up the Pinecone: 

1. Sign up on https://www.pinecone.io/
2. create an index with dim=768, metric=cosine
3. Get API Key by and put it in .env file  PINECONE_API_KEY = XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [29]:
from langchain_pinecone import PineconeVectorStore

os.getenv('PINECONE_API_KEY')

index_name = "langchain-test-index"

# Connect to Pinecone index and insert the chunked docs as contents
docsearch = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

In [30]:
query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [ ]:
found_docs =  db.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

##### FAISS
```python
        pip install faiss-cpu

In [43]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('Data/state_of_the_union.txt',encoding="ISO-8859-1").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, embeddings)

In [44]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâs top legal minds, who will continue Justice Breyerâs legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâs been nominated, sheâs received a broad range of supportâfrom the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


In [45]:
faiss_docs =  db.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(faiss_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

1. One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâs top legal minds, who will continue Justice Breyerâs legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâs been nominated, sheâs received a broad range of supportâfrom the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâve set up joint patrols with Mexico and Guatemala to catch more human traffickers. 
